General Instructions

Please submit the description of the execution plan, the code, the output, and any comments.

You can use any tool/programming language/ library you wish. Please include any dependencies/ instructions required to recreate the output.
Data

The datasets that you will work with are located at
https://www.kaggle.com/usdot/flight-delays
Flights.csv contains flight data regarding 2015 US flights. Each row can be identified by (YEAR, MONTH, DAY, AIRLINE, FLIGHT_NUMBER, TAIL_NUMBER, SCHEDULED_DEPARTURE) 

For the purposes of this exercise, we will assume that all times are in the same time zone. Tasks

Task 1: We would like you to left join flights with airlines and airports using their respective IATA code. Please describe the resulting dataset ‘flights_extended’: Number of rows, null values if any. Also, please describe any cleaning processes you may find useful or necessary.

Task 2: We would like to perform an analysis in the top 10 airports in terms of departure delay. Please create a metric to rank each airport according to the average number of aircraft that departed from that airport having a DEPARTURE_DELAY > 15 mins. Please describe if such a metric would be efficient to compare airports and include any suggestion to improve such a comparison.

Task 3: We would like to find the association, if any, between these top 10 airports and the aircraft that had no previous arrival delay (ARRIVAL_DELAY < 15) on a given day but they had arrival delay > 15 mins as soon as they departed from these airports. Please create any metrics and plots and use any technique you deem necessary to indicate the potential existence of such a phenomenon.

In [46]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import plotly.express as px
import plotly.graph_objects as go
import datetime
from datetime import date, timedelta

In [47]:
#Functions
def get_dataframe_info(df):
    """
    input
       DataFrame
    output
       DataFrame Info/description (sorted)
    Warning:Table must consist of at least 1 column with numeric values and dtype during import must not be set to unicode
    """

    df_types = pd.DataFrame(df.dtypes)
    df_nulls = df.count()
    df_transposed = pd.DataFrame(df.describe()).T
    df_overview = pd.concat([df_types, df_nulls,df_transposed ], axis=1).reset_index()
    
    # Reassign column names
    col_names = ['features', 'types', 'non_null_counts','count','mean','std','min','25%','50%','75%','max']
    df_overview .columns = col_names
    df_overview ['%nulls']=(df.shape[0]-df_overview ['non_null_counts'])*100/df.shape[0]

    
    # Add this to sort
    df_overview  = df_overview.sort_values(by=["non_null_counts"], ascending=False).drop(columns=['count','25%','50%','75%'])
    
    return df_overview 

In [48]:
airports=pd.read_csv('airports.csv')
flights=pd.read_csv('flights.csv')
airlines=pd.read_csv('airlines.csv')

C:\Users\c.logaras\Anaconda3\envs\p37env\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning:

Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.



In [49]:
#Can use the established function to check for the descriptive information per df per feature. Alternatively with the use of describe() and info()
#get_dataframe_info(airports) #2 airports na lat,long
#airlines.info() #no missing values
#get_dataframe_info(flights) # missing values on multiple columns

In [50]:
'''MERGING DATASETS'''
flights_airlines_temp=flights.merge(airlines,how='left',left_on='AIRLINE',right_on='IATA_CODE').drop(columns=['AIRLINE_x','IATA_CODE']).rename(columns={"AIRLINE_y": "AIRLINE"})

flights_extended_temp=flights_airlines_temp.merge(airports,how='left',left_on='ORIGIN_AIRPORT',right_on='IATA_CODE').drop(columns=['IATA_CODE']).rename(columns={"AIRPORT": "ORIG_AIRPORT",'CITY':'ORIG_CITY','STATE':'ORIG_STATE','COUNTRY':'ORIG_COUNTRY','LATITUDE':'ORIG_LATITUDE','LONGITUDE':'ORIG_LONGITUDE'})

flights_extended=flights_extended_temp.merge(airports,how='left',left_on='DESTINATION_AIRPORT',right_on='IATA_CODE').drop(columns=['IATA_CODE']).rename(columns={"AIRPORT": "DEST_AIRPORT",'CITY':'DEST_CITY','STATE':'DEST_STATE','COUNTRY':'DEST_COUNTRY','LATITUDE':'DEST_LATITUDE','LONGITUDE':'DEST_LONGITUDE'})

del flights_airlines_temp,flights_extended_temp

get_dataframe_info(flights_extended)


,features,types,non_null_counts,mean,std,min,max,%nulls
0,YEAR,int64,5819079,2015.000000,0.000000,2015.00000,2015.00000,0.000000
8,SCHEDULED_DEPARTURE,int64,5819079,1329.602470,483.751821,1.00000,2359.00000,0.000000
19,SCHEDULED_ARRIVAL,int64,5819079,1493.808249,507.164696,1.00000,2400.00000,0.000000
16,DISTANCE,int64,5819079,822.356495,607.784287,21.00000,4983.00000,0.000000
23,CANCELLED,int64,5819079,0.015446,0.123320,0.00000,1.00000,0.000000
30,AIRLINE,object,5819079,NaN,NaN,NaN,NaN,0.000000
22,DIVERTED,int64,5819079,0.002610,0.051020,0.00000,1.00000,0.000000
1,MONTH,int64,5819079,6.524085,3.405137,1.00000,12.00000,0.000000
7,DESTINATION_AIRPORT,object,5819079,NaN,NaN,NaN,NaN,0.000000
6,ORIGIN_AIRPORT,object,5819079,NaN,NaN,NaN,NaN,0.000000


In [51]:
del airports,flights,airlines

----------
First we will address the missing values

NULL:

Total rows in dataframe=5819079 

A)CANCELLATION_REASON has 98.45% nulls. It would make sence that when there is a cancellation we can expect null in either arrivals or departures. Similarly in diverted flights.

In [52]:
temp=flights_extended.copy()
a=get_dataframe_info(temp[temp['DEPARTURE_DELAY'].isnull()])
a.set_index('features', inplace=True)
a.loc[['CANCELLATION_REASON','DEPARTURE_DELAY','ARRIVAL_DELAY','DIVERTED']]

,types,non_null_counts,mean,std,min,max,%nulls
features,,,,,,,
CANCELLATION_REASON,object,86153,NaN,NaN,NaN,NaN,0.0
DEPARTURE_DELAY,float64,0,NaN,NaN,NaN,NaN,100.0
ARRIVAL_DELAY,float64,0,NaN,NaN,NaN,NaN,100.0
DIVERTED,int64,86153,0.0,0.0,0.0,0.0,0.0


In [53]:
print('For all the case when "departure delay" is null, cancellation has actual values:')
print(temp[temp['DEPARTURE_DELAY'].isnull()]['CANCELLATION_REASON'].unique())
print( 'There are '+str(temp['CANCELLATION_REASON'].count())+ ' non null values for cancelation.Thus we expect that the rest of actual values in cancellations correspont to nulls in "arrival delays"')

For all the case when "departure delay" is null, cancellation has actual values:
['A' 'B' 'C' 'D']
There are 89884 non null values for cancelation.Thus we expect that the rest of actual values in cancellations correspont to nulls in "arrival delays"


In other words, it is verified that all 'departure delay' nulls are explained by cancellations. We can thus impute 'departure delay' as cancelled/or 0 and control for those case via the relevant collumn. On the other hand there are still 3731 non null values for cancellation. We will check if there appears to be a relationship between 'arrival delay' and cancelation/diverted.

In [54]:
#We impute for easy use the temp['DEPARTURE_DELAY']
temp.loc[(temp['DEPARTURE_DELAY'].isnull()) , 
       'DEPARTURE_DELAY'] = 'cancelled' 
temp.loc[(temp['DEPARTURE_DELAY']== 'cancelled' ) , 
       'DEPARTURE_TIME'] = 'cancelled'

In [55]:
a=get_dataframe_info(temp[temp['ARRIVAL_DELAY'].isnull()])
a.set_index('features', inplace=True)
a.loc[['CANCELLATION_REASON','DEPARTURE_DELAY','ARRIVAL_DELAY','DIVERTED']]

,types,non_null_counts,mean,std,min,max,%nulls
features,,,,,,,
CANCELLATION_REASON,object,89884,NaN,NaN,NaN,NaN,14.454036
DEPARTURE_DELAY,object,105071,NaN,NaN,NaN,NaN,0.000000
ARRIVAL_DELAY,float64,0,NaN,NaN,NaN,NaN,100.000000
DIVERTED,int64,105071,0.14454,0.351638,0.0,1.0,0.000000


In [56]:
print('The diverted can either be null or:'+str(temp[temp['ARRIVAL_DELAY'].isnull()]['DIVERTED'].unique()))
print('Thus we have to control for the cases where diverted =0 to examine if the reason behind our missing values in arrival delays is explained by cancellations/diversions')

The diverted can either be null or:[0 1]
Thus we have to control for the cases where diverted =0 to examine if the reason behind our missing values in arrival delays is explained by cancellations/diversions


In [57]:
#  (No diverted flights) ∩ (arrival delay null) ∩ (cancellation reason is not null/0)
a=get_dataframe_info(temp[(temp['ARRIVAL_DELAY'].isnull()) & (temp['DIVERTED']==0) & (temp['CANCELLATION_REASON'].notnull())])
a.set_index('features', inplace=True)
a.loc[['CANCELLATION_REASON','DEPARTURE_DELAY','ARRIVAL_DELAY','DIVERTED']]

,types,non_null_counts,mean,std,min,max,%nulls
features,,,,,,,
CANCELLATION_REASON,object,89884,NaN,NaN,NaN,NaN,0.0
DEPARTURE_DELAY,object,89884,NaN,NaN,NaN,NaN,0.0
ARRIVAL_DELAY,float64,0,NaN,NaN,NaN,NaN,100.0
DIVERTED,int64,89884,0.0,0.0,0.0,0.0,0.0


In [58]:
#  (No diverted flights) ∩ (arrival delay null) ∩ (cancellation reason is not null/0) ∩ (departure not cancelled)
a=get_dataframe_info(temp[(temp['ARRIVAL_DELAY'].isnull()) & (temp['DIVERTED']==0) & (temp['CANCELLATION_REASON'].notnull())& (temp['DEPARTURE_DELAY']!='cancelled')])
a.set_index('features', inplace=True)
a.loc[['CANCELLATION_REASON','DEPARTURE_DELAY','ARRIVAL_DELAY','DIVERTED']]

,types,non_null_counts,mean,std,min,max,%nulls
features,,,,,,,
CANCELLATION_REASON,object,3731,NaN,NaN,NaN,NaN,0.0
DEPARTURE_DELAY,object,3731,NaN,NaN,NaN,NaN,0.0
ARRIVAL_DELAY,float64,0,NaN,NaN,NaN,NaN,100.0
DIVERTED,int64,3731,0.0,0.0,0.0,0.0,0.0


It is thus verified that the 3731 cancellation non null values that did not correspont to "departure delays",are related to arrival delays.

In [59]:
print('For 89884 flights where "arrival delay" is null, cause is cancellation. The cancellation reason can be identified as:'\
              +str(temp[(temp['ARRIVAL_DELAY'].isnull()) & (temp['DIVERTED']==0) & \
                        (temp['CANCELLATION_REASON'].notnull())]\
               ['CANCELLATION_REASON'].unique()))

For 89884 flights where "arrival delay" is null, cause is cancellation. The cancellation reason can be identified as:['A' 'B' 'C' 'D']


In [60]:
# (diverted flights) ∩ (arrival delay null) ∩ (cancellation reason IS null/0) 
a=get_dataframe_info(temp[(temp['ARRIVAL_DELAY'].isnull()) & (temp['DIVERTED']==1) & (temp['CANCELLATION_REASON'].isnull())])
a.set_index('features', inplace=True)
a.loc[['CANCELLATION_REASON','DEPARTURE_DELAY','ARRIVAL_DELAY','DIVERTED']]

,types,non_null_counts,mean,std,min,max,%nulls
features,,,,,,,
CANCELLATION_REASON,object,0,NaN,NaN,NaN,NaN,100.0
DEPARTURE_DELAY,object,15187,NaN,NaN,NaN,NaN,0.0
ARRIVAL_DELAY,float64,0,NaN,NaN,NaN,NaN,100.0
DIVERTED,int64,15187,1.0,0.0,1.0,1.0,0.0


OVERAL: 105071 null in 'arrival delay' can be explained as 15187 diverted flights and 89884 cancelled flights.
We can impute missing values and control by the relevant column.

#

Having explained the afforementioned missing values in our temp dataframe we can then impute our original dataset thus:

-Fill na all departure/arrival delays and times

-Fill na diverted,cancellation reasons

-Create a column that displays whether the flight either diverted or cancelled

-Last create a column that summarize if there was any delay (>0) , either arrival or departure

In [61]:
flights_extended['delay'] = 0
flights_extended.loc[(flights_extended['DEPARTURE_DELAY']>0) | (flights_extended['ARRIVAL_DELAY']>0), 
       'delay'] = 1 

flights_extended['Div/Canc'] = 0
flights_extended.loc[(flights_extended['CANCELLATION_REASON'].notnull()) |
                       (flights_extended['DIVERTED']==1) , 
       'Div/Canc'] = 1  


flights_extended[['CANCELLATION_REASON', 'DIVERTED','DEPARTURE_DELAY','DEPARTURE_TIME','ARRIVAL_DELAY','ARRIVAL_TIME']] = flights_extended[['CANCELLATION_REASON', 'DIVERTED','DEPARTURE_DELAY','DEPARTURE_TIME','ARRIVAL_DELAY','ARRIVAL_TIME']].fillna(value=0)


In [62]:
#Then we create a pie chart to have a quick overview
colors = ['mediumturquoise','gold','darkorange','rgb(56, 75, 126)']
labels= ['no_issue','delayed','diverted/cancelled','delayed&cancelled']
values=[flights_extended[(flights_extended['delay']==0)&(flights_extended['Div/Canc']==0)]['YEAR'].count(),flights_extended[(flights_extended['delay']==1)&(flights_extended['Div/Canc']==0)]['YEAR'].count(),flights_extended[(flights_extended['Div/Canc']==1)&(flights_extended['delay']==0)]['YEAR'].count(),flights_extended[(flights_extended['Div/Canc']==1)&(flights_extended['delay']==1)]['YEAR'].count()]
        
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.update(layout_title_text='Segmentation of flights in terms of delay/cancellation')

fig.show()

#Points of interest:

-0,182% flights departure_delayed and were later either diverted or cancelled. 1,62% of the flights are cancelled/diverted. 
These are small figures in terms of percentages but of great importance conserning customer satisfaction.

-Almost in half the total flights there are delays. Considering busy airports and the queuing implications, this is of great importance in terms of productivity.

#B Delay reasoning nulls: 81.72%

In [66]:
temp=flights_extended.copy()

temp['reasons'] = np.nan
temp.loc[(temp['WEATHER_DELAY'] >0) |
         (temp['LATE_AIRCRAFT_DELAY'] >0) |
         (temp['AIRLINE_DELAY'] >0) |
         (temp['SECURITY_DELAY'] >0) |
         (temp['AIR_SYSTEM_DELAY'] >0), 
       'reasons'] = 1 
#Checking if when there is actual delay whether a reason is provided
a=get_dataframe_info(temp[temp['delay']>0])
a.set_index('features', inplace=True)
a.loc[['WEATHER_DELAY', 'LATE_AIRCRAFT_DELAY','AIRLINE_DELAY','SECURITY_DELAY','AIR_SYSTEM_DELAY','DEPARTURE_DELAY','ARRIVAL_DELAY','reasons']]

,types,non_null_counts,mean,std,min,max,%nulls
features,,,,,,,
WEATHER_DELAY,float64,1063439,2.915290,20.433336,0.0,1211.0,60.676972
LATE_AIRCRAFT_DELAY,float64,1063439,23.472838,43.197018,0.0,1331.0,60.676972
AIRLINE_DELAY,float64,1063439,18.969547,48.161642,0.0,1971.0,60.676972
SECURITY_DELAY,float64,1063439,0.076154,2.143460,0.0,573.0,60.676972
AIR_SYSTEM_DELAY,float64,1063439,13.480568,28.003679,0.0,1134.0,60.676972
DEPARTURE_DELAY,float64,2704367,24.927057,49.445211,-42.0,1988.0,0.000000
ARRIVAL_DELAY,float64,2704367,23.694547,49.745699,-81.0,1971.0,0.000000
reasons,float64,1063439,1.000000,0.000000,1.0,1.0,60.676972


In [19]:
del temp
print( 'For 60% of the flights where there was an actual delay, we are missing the reason of the delay.\
Should we wish to examine the reason of the dealy as part of our parameters we would have available only 40% of the initial dataset')

For 60% of the flights where there was an actual delay, we are missing the reason of the delay.Should we wish to examine the reason of the dealy as part of our parameters we would have available only 40% of the initial dataset


In [ ]:
#temp[(temp['delay']>0)&(temp['reasons']!=1)]
fig = px.histogram(temp[(temp['delay']>0)&(temp['reasons']!=1)], x="DEPARTURE_DELAY")
fig.show()

Furthermore, when examining the distribution of departure_delays, when there is no justification for those a small amount of occurances with significant time size are not explained. Should all those cases had a very small delay in terms of minutes we could have imputed 0. In this case such a reasoning is not valid. 

Alternatively, the feature: delay reason, might not take a value when there was a previous change in routing. This is out of the scope of our analysis and will not be further examined. 

We will drop those column as it is not part of our objective.

In [20]:
flights_extended.drop(['WEATHER_DELAY', 'LATE_AIRCRAFT_DELAY','AIRLINE_DELAY','SECURITY_DELAY','AIR_SYSTEM_DELAY']\
                      , axis=1,inplace=True)

C)ORIG/DEST_AIRPORT/ORIG/DEST_LATITUDE  have 8.522716% nulls

In [21]:
print('The format of airports for which we do not have data is as follows:'+str(flights_extended[flights_extended['ORIG_AIRPORT'].isnull()]['ORIGIN_AIRPORT'].unique()[1:10]))
a=flights_extended[flights_extended['ORIG_AIRPORT'].isnull()]['ORIGIN_AIRPORT'].unique().tolist()
b=flights_extended[flights_extended['DEST_AIRPORT'].isnull()]['DESTINATION_AIRPORT'].unique().tolist()
a.extend(b)
c=len(set(a))
print('In total there are '+str(c)+' non IATA airport codes')
a=flights_extended[flights_extended['ORIG_AIRPORT'].notnull()]['ORIGIN_AIRPORT'].unique().tolist()
b=flights_extended[flights_extended['DEST_AIRPORT'].notnull()]['DESTINATION_AIRPORT'].unique().tolist()
a.extend(b)
d=len(set(a)) # i overwite variable to save RAM
print('In total there are '+str(d)+' IATA airport codes')

del a,b,c

The format of airports for which we do not have data is as follows:['14771' '12889' '12892' '14869' '10299' '11292' '14107' '11630' '10732']
In total there are 609 non IATA airport codes
In total there are 322 IATA airport codes


In [ ]:
colors = ['mediumturquoise','gold']
labels= ['No_IATA_CODE','IATA_CODE']
values=[flights_extended[(flights_extended['DEST_AIRPORT'].isnull())|(flights_extended['ORIG_AIRPORT'].isnull())]['YEAR'].count(),flights_extended[(flights_extended['DEST_AIRPORT'].notnull())|(flights_extended['ORIG_AIRPORT'].notnull())]['YEAR'].count()]
text=['609 airports','322 airports']
        
fig = go.Figure(data=[go.Pie(labels=labels, values=values,text=text)])
fig.update_traces(hoverinfo='label+text', textinfo='percent+value', textfont_size=10,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.update(layout_title_text='Flights with/out aiport IATA code')

fig.show()


We identified the reason of the missing values:

The airoport coding is numerical. These do not correspond to our airport dataset (IATA codes are aplhanumerical).
We suspect that they might be ISO 3166-1 numeric, and refer to countries with no latin alphabet.

In case we require to create a map in our analysis these will be ignored, but do not otherwise affect significanlty our analysis: 
The nulls are 9% of our dataset and refer to 608 airports. 
The valid 91% of our dataset is compromised of 322 known airports.

We will not drop those flights as we expect that in terms of the top 10 airports with delays, the cases with unknown airport code will not arrise.

-Representation of the bussiest aiports in term of (1)inbound and 2() outbound flights.Size represent the number of flights. ( airports with unknown code are not included)

In [23]:
aggregated=flights_extended.groupby(['DESTINATION_AIRPORT','DEST_AIRPORT']).agg({'DEST_LATITUDE':pd.Series.unique,'DEST_LONGITUDE':[pd.Series.unique,'size']}).reset_index()
aggregated.columns = aggregated.columns.droplevel(0)
aggregated.columns = ['Inbound airport code','Inbound airport', 'LATITUDE', 'LONGITUDE', 'count_flights',]
fig = px.scatter_geo(aggregated, lat="LATITUDE", lon="LONGITUDE", size="count_flights",
                   size_max=15,hover_name='Inbound airport')
fig.update(layout_title_text='Map of aiports in terms of inbound volume')
fig.show()

In [ ]:
aggregated=flights_extended.groupby(['ORIGIN_AIRPORT','ORIG_AIRPORT']).agg({'ORIG_LATITUDE':pd.Series.unique,'ORIG_LONGITUDE':[pd.Series.unique,'size']}).reset_index()
aggregated.columns = aggregated.columns.droplevel(0)
aggregated.columns = ['Outbound airport code','Outbount airport', 'LATITUDE', 'LONGITUDE', 'count_flights']
fig = px.scatter_geo(aggregated, lat="LATITUDE", lon="LONGITUDE", size="count_flights",
                   size_max=15,hover_name='Outbount airport')
fig.update(layout_title_text='Map of aiports in terms of outbound volume')
fig.show()

As expected, no significant difference appears conserning inbound/outbound volumes at first glance.

#D) 'TAXI_IN','WHEELS_ON','WHEELS_OFF','SCHEDULED_TIME','TAXI_OUT','TAIL_NUMBER','ELAPSED_TIME' nulls

In [25]:
a=get_dataframe_info(flights_extended[flights_extended['Div/Canc']!=1])
a.set_index('features', inplace=True)
a.loc[['TAXI_IN','WHEELS_ON','WHEELS_OFF','SCHEDULED_TIME','TAXI_OUT','TAIL_NUMBER','ELAPSED_TIME']]

,types,non_null_counts,mean,std,min,max,%nulls
features,,,,,,,
TAXI_IN,float64,5714008,7.429063,5.618951,1.0,248.0,0.0
WHEELS_ON,float64,5714008,1471.319332,521.868240,1.0,2400.0,0.0
WHEELS_OFF,float64,5714008,1357.099048,498.023745,1.0,2400.0,0.0
SCHEDULED_TIME,float64,5714008,141.893974,75.313998,18.0,718.0,0.0
TAXI_OUT,float64,5714008,16.065498,8.882449,1.0,225.0,0.0
TAIL_NUMBER,object,5714008,NaN,NaN,NaN,NaN,0.0
ELAPSED_TIME,float64,5714008,137.006189,74.211072,14.0,766.0,0.0


Missing values created in the aforementioned columns,are due to cancellations or diversions of flights. Since in our analysis we are required to examine how delays are affected and caused, but not cancellations or diversions, all these cases will be excluded. In othere words we will drop all the cases where flights were cancelled or diverted

In [26]:
flights_extended=flights_extended[flights_extended['Div/Canc']!=1]

We will drop all the columns that will not be used further in our analysis to decrease size of data in RAM

In [27]:
flights_extended.drop(['CANCELLATION_REASON', 'ORIG_CITY','ORIG_STATE','ORIG_COUNTRY','ORIG_LATITUDE','ORIG_LONGITUDE']\
                      , axis=1,inplace=True)
flights_extended.drop(['DEST_CITY', 'DEST_STATE','DEST_COUNTRY','DEST_LATITUDE','DEST_LONGITUDE']\
                      , axis=1,inplace=True)

----------------

DATA cleaning

Now that we have handled the missing values, we must correct the date,time information:

In [28]:
#Create DATE
flights_extended['DATE'] = pd.to_datetime(flights_extended[['YEAR','MONTH', 'DAY']], format='%Y%m%d')
flights_extended['DATE'] = flights_extended['DATE'].dt.date
#flights_extended.loc[:, ['DATE','YEAR','MONTH', 'DAY']] # used to check tranformation

In [29]:
def format_time(myst):
    '''convert the 'HHMM' string to datetime.time'''
    if pd.isnull(myst):
        return np.nan
    else:
        if myst == 2400: myst = 0
        myst = "{0:04d}".format(int(myst))
        hour = datetime.time(int(myst[0:2]), int(myst[2:4]))
        return hour

Before we transform time features, we will create a new feature that examines whether arrival was on the same CALENDAR DAY as the schedule departure

In [30]:
#departure time < actual arrival time  same date
flights_extended['same day arrival'] = np.nan
flights_extended.loc[(flights_extended['SCHEDULED_DEPARTURE']>flights_extended['ARRIVAL_TIME'])  , 
       'same day arrival'] = 0 
flights_extended.loc[(flights_extended['SCHEDULED_DEPARTURE']<flights_extended['ARRIVAL_TIME'])  , 
       'same day arrival'] = 1
#Warning the above code would fale if the flight arrival was > 24h after scheduled departure. 
#Since these are domestic flights we do not expect that to be the case

#The way we calculate the difference

duration = datetime.combine(date.min, flights_extended['DEPARTURE_TIME'][1]) - datetime.combine(date.min, flights_extended['SCHEDULED_DEPARTURE'][1])


In [31]:
#We transform all time related features
flights_extended['WHEELS_ON'] = flights_extended['WHEELS_ON'].apply(format_time)
flights_extended['SCHEDULED_ARRIVAL'] = flights_extended['SCHEDULED_ARRIVAL'].apply(format_time)
flights_extended['ARRIVAL_TIME'] = flights_extended['ARRIVAL_TIME'].apply(format_time)

In [32]:
flights_extended['WHEELS_OFF'] = flights_extended['WHEELS_OFF'].apply(format_time)

In [33]:
flights_extended['DEPARTURE_TIME'] = flights_extended['DEPARTURE_TIME'].apply(format_time) 

In [34]:
flights_extended['SCHEDULED_DEPARTURE'] = flights_extended['SCHEDULED_DEPARTURE'].apply(format_time)

In [35]:
#We combine the date and time for Scheduled departure and arrival
flights_extended['SCHEDULED_DEPARTURE_DT']=pd.to_datetime(flights_extended['DATE'].astype(str) + flights_extended['SCHEDULED_DEPARTURE'].astype(str),format = '%Y-%m-%d%H:%M:%S')
flights_extended['ARRIVAL_TIME_DT']=pd.to_datetime(flights_extended['DATE'].astype(str) + flights_extended['ARRIVAL_TIME'].astype(str),format = '%Y-%m-%d%H:%M:%S')


In [36]:
#We create a new feature for acture arrival that correct for the date, when arrival was not on same day
flights_extended['ACTUAL_ARRIVAL_TIME']=flights_extended['ARRIVAL_TIME_DT']
flights_extended.loc[flights_extended['same day arrival'] ==0, 'ACTUAL_ARRIVAL_TIME'] +=timedelta(days=1)

In [37]:
#We sort by tail number and actual arrival as it will be used in our analysis later
flights_extended.sort_values(by=["TAIL_NUMBER",'ACTUAL_ARRIVAL_TIME'],inplace=True)

In [38]:
# We create the lagged arrival information.This way we can identify departure delay do to previous late arrival
flights_extended['previous_arrival_time'] = flights_extended.groupby('TAIL_NUMBER')['ACTUAL_ARRIVAL_TIME'].shift(1)
flights_extended['previous_arrival_delay'] = flights_extended.groupby('TAIL_NUMBER')['ARRIVAL_DELAY'].shift(1)

# We will not fill NA's in-place from the 'previous arrival' column, but will drop he first occurance when needed. Should we choose to indeed fillna we would:
#flights_extended['previous_arrival_time'].fillna(flights_extended['ACTUAL_ARRIVAL_TIME'], inplace=True)
#flights_extended['previous_arrival_delay'].fillna(flights_extended['ARRIVAL_DELAY'], inplace=True)                                                                                     

#We last create a new feature that will be used to discard the previous_arrival_delay information.
The reasoning is :
    
    -If a craft arrives delayed in an airport, but between the time of arrival and next departure there is adequate time for all turnaround procedures, the previous delay should be considered null
    -A typical turnaround time is 90m, if part of the procedure is refuelling ( not always necessary )
    -For the case where the time between previous_arrival and scheduled_departure is more than 2h, we will not take into consideration the previous arrival delay.

In [39]:
flights_extended['arr_depart_diff']=np.nan                  
flights_extended['arr_depart_diff']=pd.to_timedelta(flights_extended['SCHEDULED_DEPARTURE_DT']-flights_extended['previous_arrival_time'])/pd.Timedelta('60s')


In [41]:
flights_extended.loc[flights_extended['arr_depart_diff'] >120 , 'previous_arrival_delay'] =0

In [42]:
flights_extended=flights_extended[['DATE', 'DAY_OF_WEEK', 'FLIGHT_NUMBER', 'TAIL_NUMBER',
       'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'previous_arrival_time','previous_arrival_delay',
        'SCHEDULED_DEPARTURE_DT','arr_depart_diff','DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
       'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 
        'WHEELS_ON','TAXI_IN', 'SCHEDULED_ARRIVAL', 'ACTUAL_ARRIVAL_TIME', 'ARRIVAL_DELAY',
        'AIRLINE', 'ORIG_AIRPORT', 'DEST_AIRPORT',
       'delay', 'Div/Canc','arr_depart_diff' ]]

In [43]:
#Checking validity of table
flights_extended[flights_extended['TAIL_NUMBER']=='N3KUAA'][['TAIL_NUMBER','previous_arrival_time','previous_arrival_delay','SCHEDULED_DEPARTURE_DT','arr_depart_diff','ACTUAL_ARRIVAL_TIME','ARRIVAL_DELAY']].tail(10) 


,TAIL_NUMBER,previous_arrival_time,previous_arrival_delay,SCHEDULED_DEPARTURE_DT,arr_depart_diff,arr_depart_diff,ACTUAL_ARRIVAL_TIME,ARRIVAL_DELAY
5765635,N3KUAA,2015-12-28 12:39:00,109.0,2015-12-28 13:45:00,66.0,66.0,2015-12-28 14:29:00,-16.0
5767364,N3KUAA,2015-12-28 14:29:00,-16.0,2015-12-28 15:30:00,61.0,61.0,2015-12-28 23:55:00,166.0
5775334,N3KUAA,2015-12-28 23:55:00,0.0,2015-12-29 07:00:00,425.0,425.0,2015-12-29 10:40:00,21.0
5779297,N3KUAA,2015-12-29 10:40:00,21.0,2015-12-29 11:05:00,25.0,25.0,2015-12-29 15:32:00,55.0
5783554,N3KUAA,2015-12-29 15:32:00,55.0,2015-12-29 15:30:00,-2.0,-2.0,2015-12-29 19:46:00,49.0
5787994,N3KUAA,2015-12-29 19:46:00,49.0,2015-12-29 20:10:00,24.0,24.0,2015-12-29 22:48:00,49.0
5801482,N3KUAA,2015-12-29 22:48:00,0.0,2015-12-30 17:20:00,1112.0,1112.0,2015-12-30 20:59:00,12.0
5811119,N3KUAA,2015-12-30 20:59:00,0.0,2015-12-31 11:03:00,844.0,844.0,2015-12-31 16:07:00,-28.0
5816520,N3KUAA,2015-12-31 16:07:00,-28.0,2015-12-31 17:25:00,78.0,78.0,2015-12-31 19:39:00,-12.0
5818619,N3KUAA,2015-12-31 19:39:00,-12.0,2015-12-31 21:15:00,96.0,96.0,2016-01-01 05:59:00,6.0


In [44]:
#We export to a new zip file which we can use in our analysis. 
flights_extended.to_csv('cleaned data.zip')

-----------